<a href="https://colab.research.google.com/github/AgustinGurvich/TMD/blob/main/TMD_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Estimaciones de error

In [ ]:
#---------------------------------------------------------------------------
#random forest error estimation (OOB) for greedy search
#---------------------------------------------------------------------------
rf.est <- function(x.train,y,equalize.classes=TRUE,tot.trees=500,mtry=0)
{
	if(mtry<1) mtry<-floor(sqrt(dim(x.train)[2]))
	prop.samples<-table(y)
	if(equalize.classes) prop.samples<-rep(min(prop.samples),length(prop.samples))
	return( randomForest(x.train,y,mtry=mtry,ntree=tot.trees,sampsize=prop.samples)$err.rate[tot.trees] )
}

#---------------------------------------------------------------------------
#LDA error estimation (LOO) for greedy search
#---------------------------------------------------------------------------
lda.est <- function(x.train,y)
{
	m.lda <- lda(x.train,y,CV=TRUE)
	return(error.rate( y , m.lda$class))
}
error.rate <- function(dataA, dataB) sum( dataA != dataB ) / length(dataB)

#---------------------------------------------------------------------------
#SVM error estimation (internal CV) for greedy search
#---------------------------------------------------------------------------
svm.est <- function(x.train,y,type="C-svc",kernel="vanilladot",C=1,cross = 4)
{
	return ( ksvm(x.train, y, type=type,kernel=kernel,C=C,cross = cross)@cross )
}

#---------------------------------------------------------------------------
#random forest error estimation (OOB) for greedy search
#---------------------------------------------------------------------------
rf.est <- function(x.train,y,equalize.classes=TRUE,tot.trees=500,mtry=0)
{
	if(mtry<1) mtry<-floor(sqrt(dim(x.train)[2]))
	prop.samples<-table(y)
	if(equalize.classes) prop.samples<-rep(min(prop.samples),length(prop.samples))
	return( randomForest(x.train,y,mtry=mtry,ntree=tot.trees,sampsize=prop.samples)$err.rate[tot.trees] )
}

#---------------------------------------------------------------------------
#LDA error estimation (LOO) for greedy search
#---------------------------------------------------------------------------
lda.est <- function(x.train,y)
{
	m.lda <- lda(x.train,y,CV=TRUE)
	return(error.rate( y , m.lda$class))
}
error.rate <- function(dataA, dataB) sum( dataA != dataB ) / length(dataB)

#---------------------------------------------------------------------------
#SVM error estimation (internal CV) for greedy search
#---------------------------------------------------------------------------
svm.est <- function(x.train,y,type="C-svc",kernel="vanilladot",C=1,cross = 4)
{
	return ( ksvm(x.train, y, type=type,kernel=kernel,C=C,cross = cross)@cross )
}


Codigo para el wrapper greedy forward

In [ ]:
#-------------------------------------------------------------------------------------
# AVISO: este codigo esta adaptado de un paquete mayor. 
# No es optimo y tiene cosas inutiles para nosotros. Es un ejemplo nada mas
#
#general forward greedy selection function
#input:
# x,y inputs and targets
# method is an external function that estimates classification error with a given model
# ... parameters for method
#output:
#ordered.names.list <- nombre de las variables ordenadas de la mas importante a la menos
#ordered.features.list <-numero de orden inicial de las variables, con el mismo orden
#importance <- importancia de cada variables en el mismo orden

#-------------------------------------------------------------------------------------
forward.ranking <- function(x,y,method,verbosity=0,... )
{

	max.feat<-dim(x)[2] #Cantidad de variables (digamosle k)
	num.feat<-1 #Cantidad inicial de variables
	list.feat<-1:max.feat #Sequencia de 1 a k donde guardo las variables mas importantes

	#ranking inicial: elijo la variable con menor error de prediccion
  x.train<-matrix(0,dim(x)[1],1) #Columna de 0 de tamaño n 
	class.error<-double(max.feat) #Aca voy a guardar todos los errores
	#para cada i, creo el dataset con esa variable sola, entreno un modelo y le mido el error, que lo guardo en class.error[i]
	for(i in 1:max.feat){
		x.train[,1]<-x[,i] #Lleno la variable donde guardo los datos de train con la columna de la variable que me importa
		class.error[i] <- do.call(method, c(list(x.train, y), list(...)) ) # Entreno un modelo y guardo el error
	}
	#guardo la variable con minimo error como primera. Guardo una lista keep.feat con las que me quedan para seguir eligiendo.
	list.feat[1]<-which.min(class.error) #Aca quedan ordenadas por importancia
	keep.feat<-sort(class.error,decreasing=FALSE,index=T)$ix[-1] #Aca quedan las que tengo que probar
	#armo un dataset con las variables que ya elegi, para ir agregando en cada paso.
	x.prev<-x.train[,1]<-x[,list.feat[1]] #En x.train pongo la variable importante como columna, y guardo esa columna como variable en x.prev

	if(verbosity>1) cat("\nFirst feature: ",list.feat[1],"\n")

    #loop principal. A cada paso agrego todas las variables disponibles, de a una, le mido el error y me quedo con la de minimo error. Hasta llegar a meter todas.
	while(num.feat<max.feat){
    #class.error guarda el error de cada modelo. Son max.feat-num.feat modelos.
		class.error<-double(max.feat-num.feat) #Voy sacando features
		#para cada variable que me queda, la agrego al dataset del paso anterior, entreno el modelo y le mido el error.
		for(i in 1:(max.feat-num.feat)){
			x.train<-cbind(x.prev,x[,keep.feat[i]]) #En x.prev estaban las variables que me servian, le agrego como columna la nueva variable 
			class.error[i] <- do.call(method, c(list(x.train, y), list(...)) )
		}
		if(verbosity>2) cat("\nFeatures:\n",keep.feat,"\nErrors:\n",class.error)
		#me quedo con el modelo de minimo error, guardo ese feature en la lista de las elegidas, lo saco de la lista de las que quedan, y actualizo el dataset de partida de la iteracion.
		best.index<-which.min(class.error)
		list.feat[num.feat+1]<-keep.feat[best.index]
		if(verbosity>1) cat("\n---------\nStep ",1+num.feat,"\nFeature ",best.index)

		keep.feat<-keep.feat[-best.index]
		if(verbosity>2) cat("\nNew search list: ",keep.feat)
		num.feat<-num.feat+1
		x.prev<-x[,list.feat[1:num.feat]] #Como en list.feat estaban las que me servian, ordenadas, las guardo en x.prev
	}

	print(list.feat)
	search.names<-colnames(x)[list.feat]
	#le asigno a cada feature una importacia proporcional al orden en que lo seleccionamos
	imp<-(max.feat:1)/max.feat
	names(imp)<-search.names

	if(verbosity>1){
		cat("\n---------\nFinal ranking ",num.feat," features.")
		cat("\nFeatures: ",search.names,"\n")
	}

 	return( list(ordered.names.list=search.names,ordered.features.list=list.feat,importance=imp) )

}

Codigo para el wrapper greedy backward

In [ ]:
backward.ranking <- function(x,y,method,verbosity=0,... )
{

	max.feat<-dim(x)[2] #Cantidad de variables (digamosle k)
	num.feat<-0 #Cantidad de variables que saque (ninguna)
	list.feat<-1:max.feat #Sequencia de 1 a k donde guardo las variables mas importantes

	#ranking inicial: entreno un modelo completo y le mido el error
    x.train<-matrix(0,dim(x)[1],max.feat - num.feat)  
	x.train<-x #El primer entrenamiento se hace con todo el dataset
	previous.error <- do.call(method, c(list(x.train, y), list(...)) ) # Entreno un modelo completo y guardo el error

    class.error<-double(max.feat)
    for(i in 1:max.feat){
		x.train<-x[,-i]
		class.error[i] <- do.call(method, c(list(x.train, y), list(...)) )
        class.error[i] <- abs(class.error[i] - previous.error) #Diferencia con el error anterior
    }


	#Saco la variable que menos error me agrego (no era tan importante). Guardo una lista keep.feat con las que me quedan para seguir sacando.
	min.error.index <- which.min(class.error) #La que menor error deja es la menos importante (no afecto que la saque) 
    previous.error <- class.error[min.error.index] #El modelo adecuado es este
    list.feat[1]<-min.error.index #Aca quedan ordenadas por menor importancia
	keep.feat<-sort(class.error,decreasing=FALSE,index=T)$ix[-1] #Aca quedan las que tengo que probar
	#Saco del dataset las variables que ya probe
	x.prev<-x.train[,-1]<-x[,-list.feat[1]] #En x.train pongo la variable importante como columna, y guardo esa columna como variable en x.prev
    num.feat<-num.feat+1

	if(verbosity>1) cat("\nFirst iteration completed. Removed feat: ",list.feat[1],"\n")

    #loop principal. A cada paso saco variables, de a una, le mido el error y elimino la de minimo error. Hasta llegar a sacar todas.
	while(num.feat<max.feat){
    print(num.feat)
    print(keep.feat)
        #class.error guarda el error de cada modelo. Son max.feat-num.feat modelos.
		class.error<-double(max.feat-num.feat) #Voy sacando features
		#para cada variable que me queda, la saco del dataset del paso anterior, entreno el modelo y le mido el error.
		for(i in 1:(max.feat-num.feat)){
			x.train<-x.prev[,-keep.feat[i]] #En x.prev estaban las variables que me servian, le saco como columna la nueva variable 
			class.error[i] <- do.call(method, c(list(x.train, y), list(...)) )
			class.error[i] <- abs(class.error[i] - previous.error) #Diferencia con el error anterio
		}
		if(verbosity>2) cat("\nFeatures:\n",keep.feat,"\nErrors:\n",class.error)
		#me quedo con el modelo de minimo error, guardo ese feature en la lista de las elegidas, lo saco de la lista de las que quedan, y actualizo el dataset de partida de la iteracion.
		min.error.index<-which.min(class.error)
		previous.error <- class.error[min.error.index]
		list.feat[num.feat+1]<-keep.feat[min.error.index]
		if(verbosity>1) cat("\n---------\nStep ",1+num.feat,"\nFeature ",min.error.index)

		keep.feat<-keep.feat[-min.error.index]
		if(verbosity>2) cat("\nNew search list: ",keep.feat)
		num.feat<-num.feat+1
		x.prev<-x.prev[,-list.feat[num.feat]] #Como en list.feat estaban las que me servian, ordenadas, las guardo en x.prev
	}


	search.names<-colnames(x)[list.feat]
	#le asigno a cada feature una importacia proporcional al orden en que lo seleccionamos
	imp<-(max.feat:1)/max.feat
	names(imp)<-search.names

	if(verbosity>1){
		cat("\n---------\nFinal ranking ",num.feat," features.")
		cat("\nFeatures: ",search.names,"\n")
	}

 	return( list(ordered.names.list=search.names,ordered.features.list=list.feat,importance=imp) )

}

Codigo para el filter con test no-paramétrico

In [5]:
no.parametrico <- function(x,y){
  var.count <- dim(x)[2]
  stat <- matrix(ncol = var.count, nrow = 1)
  for (i in 1:var.count){
    var<-x[,i]
    stat[1,i] <- kruskal.test(var,y)$statistic
  }
  ordered.vars <- order(stat, decreasing=T) 
  var.names<-colnames(x)[ordered.vars]
  return( list(ordered.names.list=var.names,ordered.features.list=ordered.vars,importance=sort(stat,decreasing=T)) )

}

In [ ]:
Codigo para el RFE con SVM

In [ ]:
rfe <- function(x,y,method,...){
  F <- x
  p <- dim(x)[2]
  R <- matrix(ncol = p, nrox = 1)
  for(i in 1:p){
    #¿Que es eso del metodo? Guardarlo en ranked
    fIndex <- which.min(ranked) 
    R[1,p-i+1] <- fIndex
    F <- F[,-fIndex]
  }
  ordered.vars <- order(R, decreasing=T) 
  var.names<-colnames(x)[ordered.vars]
  return( list(ordered.names.list=var.names,ordered.features.list=ordered.vars,importance=sort(R,decreasing=T)) )

}

Test

In [ ]:
crea.ruido.unif<-function(n=100,d=2){
  x<-runif(2*n*d,min=-1)	#genero los datos
  dim(x)<-c(2*n,d)
  return(cbind(as.data.frame(x),y=factor(rep(c(-1,1),each=n))))	#le agrego la clase
}

#datosA
d<-10
n<-20
datos<-crea.ruido.unif(n=n,d=d)

#tomar 50% de los datos al azar, y hacer que la clase sea el signo de la 8 variable
shuffle<-sample(1:dim(datos)[1])
sub<-shuffle[1:dim(datos)[1]*0.5]
datos[sub,d+1]<-sign(datos[sub,8])
#tomar 20% de los datos al azar (fuera de los anteriores), y hacer que la clase sea el signo de la 6 variable
sub<-shuffle[(dim(datos)[1]*0.5):(dim(datos)[1]*0.7)]
datos[sub,d+1]<-sign(datos[sub,6])
#tomar 10% de los datos al azar, y hacer que la clase sea el signo de la 4 variable
sub<-shuffle[(dim(datos)[1]*0.7):(dim(datos)[1]*0.8)]
datos[sub,d+1]<-sign(datos[sub,4])
#tomar 5% de los datos al azar, y hacer que la clase sea el signo de la 2 variable
sub<-shuffle[(dim(datos)[1]*0.8):(dim(datos)[1]*0.85)]
datos[sub,d+1]<-sign(datos[sub,2])
datos[,d+1]<-factor(datos[,d+1])

datosA<-datos
#print(dim(datosA))
#print(matrix(0,dim(datosA)[1],1))
 1:(10-3)

[1] 1 2 3 4 5 6 7

In [6]:
#install.packages("randomForest", dependencies = T)
#library(randomForest)
#install.packages("kernlab", dependencies = T)
#library(kernlab)
#library(MASS)

#demo: aplicar el wrapper a los datos de iris
data(iris)
#FORW.rf <-forward.ranking(iris[,-5],iris[,5],method="rf.est" ,tot.trees=100,equalize.classes=F)
#FORW.lda<-forward.ranking(iris[,-5],iris[,5],method="lda.est")
#print(FORW.lda)

#BACK.rf <-backward.ranking(iris[,-5],iris[,5],method="rf.est" ,tot.trees=100,equalize.classes=F)
#BACK.lda<-backward.ranking(iris[,-5],iris[,5],method="lda.est")
no.parametrico(iris[,-5],iris[,5])

$ordered.names.list
[1] "Petal.Width"  "Petal.Length" "Sepal.Length" "Sepal.Width" 

$ordered.features.list
[1] 4 3 1 2

$importance
[1] 131.18538 130.41105  96.93744  63.57115

     [,1] [,2] [,3] [,4] [,5]
[1,]   NA   NA   NA   NA   NA
[1] 1 1


ERROR: ignored